In [93]:
from pynq import Overlay, allocate
import numpy as np

In [94]:
Matmul_HW = Overlay("Matmul_100MHz.bit")

In [95]:
Matmul_HW?

In [96]:
cmem_0 = Matmul_HW.cmem_0
A_MSB = Matmul_HW.axi_bram_ctrl_0
A_LSB = Matmul_HW.axi_bram_ctrl_1
B_MSB = Matmul_HW.axi_bram_ctrl_2
B_LSB = Matmul_HW.axi_bram_ctrl_3
OUTPUT = Matmul_HW.axi_bram_ctrl_4

In [97]:

def read_bfloat16_binary_file_to_list(filename):
    """
    Reads a text file containing continuous BFLOAT16 binary representations 
    and converts it to a list.

    Args:
        filename (str): The path to the text file.

    Returns:
        list of list of str: A 2D list of BFLOAT16 binary strings.
    """
    with open(filename, 'r') as f:
        # Read all lines, stripping any newline characters
        lines = f.readlines()

    # Define BFLOAT16 binary length
    binary_length = 16

    # Process each line
    result = []
    for line in lines:
        # Remove newline and split into chunks of 16 bits
        chunks = [line[i:i+binary_length] for i in range(0, len(line.strip()), binary_length)]
        result.append(chunks)

    return result

In [98]:
A_test = read_bfloat16_binary_file_to_list("A_64x128_raw.txt")
B_test = read_bfloat16_binary_file_to_list("B_64x128_raw.txt")

In [99]:
print(A_test[0])
#print(B_test)

['1100000000000101', '1100000011010010', '1100000011000011', '0100000100111111', '0100000110011011', '0100000110010111', '1100000101011111', '0100000011011011', '1100000110001011', '1100000001011101', '0100000010010001', '1100000110011001', '1100000110010010', '1100000100111001', '1100000110001011', '1011111001011101', '0100000010011011', '0100000110010111', '1100000000000001', '1100000110011101', '1100000011101100', '0100000101010001', '0100000011010000', '1100000101010011', '1011111111100100', '1100000100101110', '1100000101001100', '1100000010001101', '0100000010111001', '1100000011110010', '0100000110011001', '1100000100000001', '1100000011010110', '1100000010111110', '0100000110001101', '1100000110001011', '1100000101100110', '0100000110010010', '0100000100101001', '1100000001000010', '0100000101110000', '1100000110001011', '1100000010101111', '0100000110010110', '1100000010010000', '1100000100001100', '1100000010100011', '1100000101011010', '1100000110011010', '0100000101111110',

In [100]:
def BF_to_int(input_matrix):
    int_matrix = [[int(binary, 2) for binary in row] for row in input_matrix]
    return int_matrix

In [101]:
A_test_int = BF_to_int(A_test)
B_test_int = BF_to_int(B_test)

In [102]:
print(A_test_int[0])
print(A_test_int[0][63], A_test_int[0][64])
#print(bin(A_test_int[0][0])[2:].zfill(16))
#print(bin(A_test_int[0][1])[2:].zfill(16))

[49157, 49362, 49347, 16703, 16795, 16791, 49503, 16603, 49547, 49245, 16529, 49561, 49554, 49465, 49547, 48733, 16539, 16791, 49153, 49565, 49388, 16721, 16592, 49491, 49124, 49454, 49484, 49293, 16569, 49394, 16793, 49409, 49366, 49342, 16781, 49547, 49510, 16786, 16681, 49218, 16752, 49547, 49327, 16790, 49296, 49420, 49315, 49498, 49562, 16766, 16740, 49409, 16568, 49369, 49177, 49543, 49513, 16596, 49535, 49421, 49292, 16477, 48699, 49241, 49383, 16446, 49313, 16797, 49540, 16511, 16420, 48919, 49555, 49469, 49560, 48922, 49039, 16771, 49287, 49559, 49391, 16779, 49212, 49540, 49555, 16672, 16661, 49542, 16015, 49462, 49463, 16556, 49411, 49415, 16568, 49204, 49536, 16566, 49521, 16530, 16518, 48972, 49345, 16431, 49506, 16757, 16501, 16487, 15983, 16782, 16660, 49561, 16629, 49240, 16395, 16704, 16669, 49565, 49205, 16701, 16641, 16719, 16742, 16640, 49371, 49539, 48455, 49357]
49241 49383


In [103]:
# input check (first row, 128 data)

for row_a, row_b in zip(A_test_int, B_test_int):
    A_MSB_list = row_a[:64]
    A_LSB_list = row_a[64:]
    print(A_MSB_list)
    print(' ')
    print(A_LSB_list)
    print(' ')
    
    B_MSB_list = row_b[:64]
    B_LSB_list = row_b[64:]
    print(B_MSB_list)
    print(' ')
    print(B_LSB_list)
    break


[49157, 49362, 49347, 16703, 16795, 16791, 49503, 16603, 49547, 49245, 16529, 49561, 49554, 49465, 49547, 48733, 16539, 16791, 49153, 49565, 49388, 16721, 16592, 49491, 49124, 49454, 49484, 49293, 16569, 49394, 16793, 49409, 49366, 49342, 16781, 49547, 49510, 16786, 16681, 49218, 16752, 49547, 49327, 16790, 49296, 49420, 49315, 49498, 49562, 16766, 16740, 49409, 16568, 49369, 49177, 49543, 49513, 16596, 49535, 49421, 49292, 16477, 48699, 49241]
 
[49383, 16446, 49313, 16797, 49540, 16511, 16420, 48919, 49555, 49469, 49560, 48922, 49039, 16771, 49287, 49559, 49391, 16779, 49212, 49540, 49555, 16672, 16661, 49542, 16015, 49462, 49463, 16556, 49411, 49415, 16568, 49204, 49536, 16566, 49521, 16530, 16518, 48972, 49345, 16431, 49506, 16757, 16501, 16487, 15983, 16782, 16660, 49561, 16629, 49240, 16395, 16704, 16669, 49565, 49205, 16701, 16641, 16719, 16742, 16640, 49371, 49539, 48455, 49357]
 
[49434, 49354, 16563, 16787, 16368, 16215, 49350, 16737, 16788, 16713, 49412, 49101, 16626, 16738,

In [104]:
def group_two_inputs_convert_to_int32(int_list):
    # Convert 16-bit integer to 32-bit integer
    binary_string = ''.join(format(x, '016b') for x in int_list)  # Convert to 16-bit binary string and then concatenate
    # Group by two and convert to 32-bit integer
    return [int(binary_string[i:i+32], 2) for i in range(0, len(binary_string), 32)]

In [105]:
a = group_two_inputs_convert_to_int32(A_test_int[0])
print(a)
print(bin(a[0])[2:].zfill(16))

[3221602514, 3234021695, 1100693911, 3244245211, 3247161437, 1083294105, 3247620409, 3247160925, 1083916695, 3221340573, 3236708689, 1087422803, 3219439918, 3243032717, 1085915378, 1100595457, 3235299518, 1099809163, 3244704146, 1093255234, 1097908619, 3232711062, 3230712076, 3231957338, 3248111998, 1097122049, 1085849817, 3222913415, 3244900564, 3246375181, 3230416989, 3191586905, 3236380734, 3231793565, 3246669951, 1076150039, 3247685949, 3248013082, 3213836675, 3230122391, 3236905355, 3225207172, 3247653152, 1091944838, 1049608502, 3241623724, 3238248711, 1085849652, 3246407862, 3245424786, 1082572620, 3233890351, 3244441973, 1081426023, 1047478670, 1091879321, 1089847384, 1074479424, 1092469149, 3224715581, 1090601295, 1097220352, 3235627395, 3175596237]
11000000000001011100000011010010


In [106]:
print(type(A_MSB))

<class 'pynq.overlay.DefaultIP'>


In [107]:
# Insert input Matrix A, B into four brams

row = 0 
for row_a, row_b in zip(A_test_int, B_test_int): # row = 0 ~ 63
    A_MSB_list = group_two_inputs_convert_to_int32(row_a[:64])
    A_LSB_list = group_two_inputs_convert_to_int32(row_a[64:])
    B_MSB_list = group_two_inputs_convert_to_int32(row_b[:64])
    B_LSB_list = group_two_inputs_convert_to_int32(row_b[64:])
    
    for idx, (a_, b_) in enumerate(zip(A_MSB_list, B_MSB_list)): # idx = 0 ~ 31
        A_MSB.mmio.write(row*4*32 + idx*4, a_)
        B_MSB.mmio.write(row*4*32 + idx*4, b_) 
        
    for idx, (a_, b_) in enumerate(zip(A_LSB_list, B_LSB_list)):
        A_LSB.mmio.write(row*4*32 + idx*4, a_) 
        B_LSB.mmio.write(row*4*32 + idx*4, b_)

    
    if row == 0:
        print("A_MSB_list:", A_MSB_list)
        print("Verification:")
        for idx, _ in enumerate(A_MSB_list):
            a_read = A_MSB.mmio.read(idx * 4)
            b_read = B_MSB.mmio.read(idx * 4)
            print(f"A_MSB[{idx}]: {a_read}, B_MSB[{idx}]: {b_read}")

        for idx, _ in enumerate(A_LSB_list):
            a_read = A_LSB.mmio.read(idx * 4)
            b_read = B_LSB.mmio.read(idx * 4)
            print(f"A_LSB[{idx}]: {a_read}, B_LSB[{idx}]: {b_read}")
    
    row += 1
    

A_MSB_list: [3221602514, 3234021695, 1100693911, 3244245211, 3247161437, 1083294105, 3247620409, 3247160925, 1083916695, 3221340573, 3236708689, 1087422803, 3219439918, 3243032717, 1085915378, 1100595457, 3235299518, 1099809163, 3244704146, 1093255234, 1097908619, 3232711062, 3230712076, 3231957338, 3248111998, 1097122049, 1085849817, 3222913415, 3244900564, 3246375181, 3230416989, 3191586905]
Verification:
A_MSB[0]: 3221602514, B_MSB[0]: 3239755978
A_MSB[1]: 3234021695, B_MSB[1]: 1085489555
A_MSB[2]: 1100693911, B_MSB[2]: 1072709463
A_MSB[3]: 3244245211, B_MSB[3]: 3234218337
A_MSB[4]: 3247161437, B_MSB[4]: 1100235081
A_MSB[5]: 1083294105, B_MSB[5]: 3238313933
A_MSB[6]: 3247620409, B_MSB[6]: 1089618274
A_MSB[7]: 3247160925, B_MSB[7]: 3225371009
A_MSB[8]: 1083916695, B_MSB[8]: 1090928774
A_MSB[9]: 3221340573, B_MSB[9]: 3246604606
A_MSB[10]: 3236708689, B_MSB[10]: 1050591618
A_MSB[11]: 1087422803, B_MSB[11]: 1099218940
A_MSB[12]: 3219439918, B_MSB[12]: 3225502049
A_MSB[13]: 3243032717, B

In [108]:
# Send the start signal to the hardware
cmem_0.write(0x00, 1)  # start_signal

import time
startTime = time.time()
# Wait for the operation to complete
while True:
    status = cmem_0.read(0x04)  # Status register address
    if status & 1:  # Check if the 'done' bit (bit 0) is set
        break
print(f"HW time : {(time.time() - startTime)*1000:.10f} ms")
print("Output Matrix waiting finish")

HW time : 0.5011558533 ms
Output Matrix waiting finish


In [109]:
from BF_to_FP_jupyter import BF_to_float_single, BF_to_float_2d

In [110]:
output_matrix = []

for m in range(64): # 64 rows
    row = []
    for k in range(32): # 32 cols
        matrix_element = OUTPUT.mmio.read(m*4*32 + k*4)
        matrix_element_div = bin(matrix_element)[2:].zfill(32)
        element_float = BF_to_float_single(matrix_element_div[:16])
        row.append(element_float)
        element_float_ = BF_to_float_single(matrix_element_div[16:])
        row.append(element_float_)
        
    output_matrix.append(row)
    

In [111]:
print("Matmul HW output, First row 64 elements : ", output_matrix[0])
print(len(output_matrix[0]))

Matmul HW output, First row 64 elements :  [848.0, 928.0, 1152.0, 1488.0, -1976.0, -1112.0, 2272.0, 1592.0, -80.0, -672.0, 1336.0, -136.0, -800.0, -960.0, -400.0, 104.0, -788.0, -380.0, 920.0, -760.0, -1752.0, -504.0, -1880.0, 2832.0, -1024.0, -40.0, 1640.0, -1584.0, -1464.0, -396.0, 460.0, -1888.0, -1440.0, 124.0, 720.0, -680.0, -1288.0, 1552.0, -320.0, -1336.0, -292.0, -2040.0, 780.0, 2016.0, -1152.0, 444.0, -1840.0, -576.0, -932.0, 868.0, 1864.0, 848.0, -856.0, 2272.0, -374.0, -912.0, 2288.0, 472.0, 12.0, -664.0, 912.0, -488.0, 984.0, -438.0]
64


In [112]:
################################ SW : Numpy matmul ################################

import numpy as np


A_float = BF_to_float_2d(A_test)
B_float = BF_to_float_2d(B_test)
    
A_ = np.array(A_float, dtype='float16')
B_ = np.array(B_float, dtype='float16')
startTime = time.time()
matmul_numpy = np.matmul(A_, B_.T)
print(f"SW time : {(time.time() - startTime)*1000:.10f} ms")

print("Matmul SW output, First row 64 elements : ", matmul_numpy[0])

SW time : 23.0848789215 ms
Matmul SW output, First row 64 elements :  [  861.5     934.     1172.     1441.    -1992.    -1068.     2280.
  1597.      -83.7    -676.     1347.     -128.2    -767.5    -961.5
  -398.      109.44   -799.     -394.8     937.5    -774.    -1765.
  -500.5   -1907.     2916.    -1032.      -54.06   1653.    -1469.
 -1467.     -408.2     333.8   -1903.    -1386.      113.4     725.
  -670.    -1291.     1581.     -199.    -1294.     -291.8   -2060.
   789.     2034.    -1158.      384.    -1854.     -583.     -941.
   892.5    1891.      845.5    -864.5    2304.     -377.     -929.5
  2186.      472.8      10.555  -661.      904.     -494.     1066.
  -433.2  ]
